## **Install Transformer**
This line of code installs several Python libraries required for working with NLP models and optimizing them. The exclamation mark at the start allows it to be run directly in a notebook like Google Colab. transformers provides pre-trained models such as BERT and RoBERTa for tasks like text classification and sentiment analysis, while datasets helps with accessing and managing NLP datasets. accelerate enables faster training by supporting multiple devices and mixed-precision. ray[tune] and optuna are tools for hyperparameter optimization, helping to find the best model settings. Finally, the -U flag ensures all the packages are updated to their latest versions.


In [ ]:
!pip install transformers datasets accelerate ray[tune] optuna -U

## **Setup and Installation**
```
import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import optuna # Import optuna to use its suggestion methods for random search

# Set a consistent seed for reproducibility across runs
set_seed(42)

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. For faster training, consider enabling a GPU runtime.")
```
This code sets up the environment for training a text classification model. It first imports the necessary libraries: torch for deep learning, numpy and pandas for data handling, datasets for managing NLP datasets, transformers for working with pre-trained models, sklearn for data splitting and evaluation metrics, and optuna for hyperparameter optimization. The set_seed(42) function ensures that the results are reproducible, meaning the same results can be obtained each time the code runs. The code then checks if a GPU is available for faster training; if not, it uses the CPU, which is slower but still works.

-----
## **Data Preparation**
```
try:
    df = pd.read_csv("Mental-Health-Twitter.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'Mental-Health-Twitter.csv' not found. Please upload it to your Colab environment.")
    exit()

# Filter out rows where 'post_text' is NaN or empty
df = df.dropna(subset=['post_text'])
df = df[df['post_text'].str.strip() != '']

# Rename 'label' to 'labels' for Hugging Face Trainer compatibility
df = df.rename(columns={"label": "labels"})

# Split data into training and validation sets
train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['labels'], random_state=42)
train_df = train_df.sample(n=5000, random_state=42)
eval_df = eval_df.sample(n=1000, random_state=42)

print(f"Using {len(train_df)} training samples and {len(eval_df)} evaluation samples.")
print(f"Train label distribution:\n{train_df['labels'].value_counts(normalize=True)}")
print(f"Eval label distribution:\n{eval_df['labels'].value_counts(normalize=True)}")

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df[['post_text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['post_text', 'labels']])

# Initialize Tokenizer for your specific model
MODEL_NAME = "margotwagner/roberta-psychotherapy-eval"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["post_text"], truncation=True, padding=True, max_length=128)

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])

tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
```
This code handles loading and preparing the dataset for model training. It first attempts to read the CSV file Mental-Health-Twitter.csv and stops with an error message if the file is not found. Rows with missing or empty text in the post_text column are removed, and the target column is renamed to labels to be compatible with the Hugging Face Trainer. The dataset is split into training (90%) and evaluation (10%) sets, and a smaller subset of 5,000 training samples and 1,000 evaluation samples is selected to reduce computation time. The distribution of labels in both sets is printed to ensure balance.
Next, the training and evaluation data are converted into Hugging Face Dataset objects. A pre-trained RoBERTa tokenizer is loaded, and a tokenize_function converts text into token IDs and attention masks, truncating or padding sequences to a maximum length of 128 tokens. Finally, the tokenized datasets are formatted for PyTorch, keeping only the columns needed for model training: input_ids, attention_mask, and labels.

-----
##**MODEL, METRICS, AND HYPERPARAMETER**
```
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    precision = precision_score(p.label_ids, preds, average="binary")
    recall = recall_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

def random_hp_space(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1, step=0.01)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 3)
    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": per_device_train_batch_size,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "weight_decay": weight_decay,
        "num_train_epochs": num_train_epochs,
    }
```
This code defines the model, evaluation metrics, and hyperparameter search space for training. The model_init() function loads a pre-trained RoBERTa model for sequence classification with two output labels and moves it to the selected device (GPU or CPU). The compute_metrics() function calculates key evaluation metrics—accuracy, F1-score, precision, and recall—by comparing the model’s predictions with the true labels.The random_hp_space() function defines the hyperparameter space for optimization. It specifies ranges or options for the learning rate, batch size per device, gradient accumulation steps, weight decay, and number of training epochs. These hyperparameters are then tested during training to find the combination that achieves the best model performance.

-----
## **Training Arguments**
```
training_args = TrainingArguments(
    output_dir="./random_search_results_mental_health_5gb",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none",
    num_train_epochs=3,
    warmup_steps=100,
    logging_dir="./logs_random_5gb",
    logging_steps=200,
    dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    adam_epsilon=1e-7,
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

def optuna_hp_objective(metrics):
    return metrics["eval_f1"]
```
This code sets up the training configuration and trainer for the model. The TrainingArguments define how the model will be trained and evaluated. Training and evaluation results are saved in designated folders, and the model is evaluated and saved at the end of each epoch. Only the best model based on the F1-score is retained, and mixed-precision training (fp16) is enabled if a GPU is available for faster computation. Other settings include the number of training epochs, warmup steps, logging frequency, number of CPU workers for data loading, and the Adam optimizer’s epsilon value. The Trainer class from Hugging Face is then initialized with the model, training arguments, tokenized datasets, evaluation metrics, and tokenizer. Finally, the optuna_hp_objective() function is defined to return the evaluation F1-score, which guides the hyperparameter optimization process.

-----
## **Execution of Random Search**
```
print("\n--- Starting Random Search (using Optuna backend) ---")
print(f"Optimizing for '{training_args.metric_for_best_model}' score...")

NUM_RANDOM_TRIALS = 30

print(f"Number of random trials to run: {NUM_RANDOM_TRIALS}")
print("NOTE: Batch sizes are kept low and gradient accumulation is used to manage 5GB GPU memory.")
print("      Dataset size has also been reduced for quicker iteration.")

best_trial = trainer.hyperparameter_search(
    backend="optuna",
    hp_space=random_hp_space,
    direction="maximize",
    n_trials=NUM_RANDOM_TRIALS,
    compute_objective=optuna_hp_objective,
)

print("\n--- Random Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
    print(f"\nBest Metrics (on evaluation set): {best_trial.metrics}")
else:
    print("Search failed or no best trial found.")

print("\n--- Final Step: Train a model with the best hyperparameters ---")
if best_trial:
    final_training_args = TrainingArguments(
        output_dir="./final_model_mental_health_5gb_random",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),
        report_to="none",
        num_train_epochs=best_hps["num_train_epochs"],
        per_device_train_batch_size=best_hps["per_device_train_batch_size"],
        gradient_accumulation_steps=best_hps["gradient_accumulation_steps"],
        learning_rate=best_hps["learning_rate"],
        weight_decay=best_hps["weight_decay"],
        warmup_steps=100,
        logging_dir="./final_logs_random_5gb",
        logging_steps=200,
        dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
        adam_epsilon=1e-7,
    )

    final_trainer = Trainer(
        model_init=model_init,
        args=final_training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("\nTraining final model with best hyperparameters from Random Search (5GB GPU config)...")
    final_trainer.train()

    print("\nFinal model training complete. Best model saved to './final_model_mental_health_5gb_random'.")
    metrics = final_trainer.evaluate()
    print(f"Evaluation metrics of the final model: {metrics}")
else:
    print("No best hyperparameters found, skipping final model training.")
```
This code performs random hyperparameter search using Optuna and trains the final model with the best parameters found. It begins by announcing the start of the search, specifying that the F1-score will be used to determine the best model. A total of 30 random trials are conducted, with smaller batch sizes and gradient accumulation used to manage GPU memory constraints, and a reduced dataset size for quicker experimentation. The trainer.hyperparameter_search() method runs the random search according to the random_hp_space() function, and the best hyperparameters and evaluation metrics are printed. Once the optimal parameters are identified, new TrainingArguments are created using these values, including learning rate, batch size, number of epochs, weight decay, and gradient accumulation steps. A new Trainer is then initialized, and the final model is trained on the tokenized training dataset. After training, the model is evaluated on the validation set, and its performance metrics are displayed. The trained model is saved in the folder ./final_model_mental_health_5gb_random for future use.

In [ ]:
# 1. SETUP AND INSTALLATION
# Run this command first in your Colab notebook:
# !pip install transformers datasets accelerate ray[tune] optuna pandas -U

import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import optuna # Import optuna to use its suggestion methods for random search

# Set a consistent seed for reproducibility across runs
set_seed(42)

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. For faster training, consider enabling a GPU runtime.")

# --- 2. DATA PREPARATION (Using your Mental-Health-Twitter.csv) ---

# Upload 'Mental-Health-Twitter.csv' to your Colab environment
# Example: from google.colab import files
#          files.upload() # Then select your file

# Load your dataset
try:
    df = pd.read_csv("Mental-Health-Twitter.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'Mental-Health-Twitter.csv' not found. Please upload it to your Colab environment.")
    exit()

# Filter out rows where 'post_text' is NaN or empty
df = df.dropna(subset=['post_text'])
df = df[df['post_text'].str.strip() != '']

# Rename 'label' to 'labels' for Hugging Face Trainer compatibility
df = df.rename(columns={"label": "labels"})

# Split data into training and validation sets
train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['labels'], random_state=42)
train_df = train_df.sample(n=5000, random_state=42)
eval_df = eval_df.sample(n=1000, random_state=42)

print(f"Using {len(train_df)} training samples and {len(eval_df)} evaluation samples.")
print(f"Train label distribution:\n{train_df['labels'].value_counts(normalize=True)}")
print(f"Eval label distribution:\n{eval_df['labels'].value_counts(normalize=True)}")

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df[['post_text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['post_text', 'labels']])

# Initialize Tokenizer for your specific model
MODEL_NAME = "margotwagner/roberta-psychotherapy-eval"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["post_text"], truncation=True, padding=True, max_length=128)

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])

tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])


# --- 3. MODEL, METRICS, AND HYPERPARAMETER DEFINITION ---

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    precision = precision_score(p.label_ids, preds, average="binary")
    recall = recall_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

def random_hp_space(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1, step=0.01)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 3)
    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": per_device_train_batch_size,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "weight_decay": weight_decay,
        "num_train_epochs": num_train_epochs,
    }


# --- 4. TRAINING ARGUMENTS (Fixed for all runs) ---
training_args = TrainingArguments(
    output_dir="./random_search_results_mental_health_5gb",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none",
    num_train_epochs=3,
    warmup_steps=100,
    logging_dir="./logs_random_5gb",
    logging_steps=200,
    dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    adam_epsilon=1e-7,
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

def optuna_hp_objective(metrics):
    return metrics["eval_f1"]


# --- 5. EXECUTION OF RANDOM SEARCH ---
print("\n--- Starting Random Search (using Optuna backend) ---")
print(f"Optimizing for '{training_args.metric_for_best_model}' score...")

NUM_RANDOM_TRIALS = 30

print(f"Number of random trials to run: {NUM_RANDOM_TRIALS}")
print("NOTE: Batch sizes are kept low and gradient accumulation is used to manage 5GB GPU memory.")
print("      Dataset size has also been reduced for quicker iteration.")

best_trial = trainer.hyperparameter_search(
    backend="optuna",
    hp_space=random_hp_space,
    direction="maximize",
    n_trials=NUM_RANDOM_TRIALS,
    compute_objective=optuna_hp_objective,
)

print("\n--- Random Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
    print(f"\nBest Metrics (on evaluation set): {best_trial.metrics}")
else:
    print("Search failed or no best trial found.")

print("\n--- Final Step: Train a model with the best hyperparameters ---")
if best_trial:
    final_training_args = TrainingArguments(
        output_dir="./final_model_mental_health_5gb_random",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),
        report_to="none",
        num_train_epochs=best_hps["num_train_epochs"],
        per_device_train_batch_size=best_hps["per_device_train_batch_size"],
        gradient_accumulation_steps=best_hps["gradient_accumulation_steps"],
        learning_rate=best_hps["learning_rate"],
        weight_decay=best_hps["weight_decay"],
        warmup_steps=100,
        logging_dir="./final_logs_random_5gb",
        logging_steps=200,
        dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
        adam_epsilon=1e-7,
    )

    final_trainer = Trainer(
        model_init=model_init,
        args=final_training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("\nTraining final model with best hyperparameters from Random Search (5GB GPU config)...")
    final_trainer.train()

    print("\nFinal model training complete. Best model saved to './final_model_mental_health_5gb_random'.")
    metrics = final_trainer.evaluate()
    print(f"Evaluation metrics of the final model: {metrics}")
else:
    print("No best hyperparameters found, skipping final model training.")

Using GPU: Tesla T4
Dataset loaded successfully.
Using 5000 training samples and 1000 evaluation samples.
Train label distribution:
labels
1    0.5
0    0.5
Name: proportion, dtype: float64
Eval label distribution:
labels
0    0.507
1    0.493
Name: proportion, dtype: float64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

[I 2025-11-08 08:18:53,670] Trial 28 finished with value: 0.880641925777332 and parameters: {'learning_rate': 4.24946942200687e-05, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 4, 'weight_decay': 0.05, 'num_train_epochs': 2}. Best is trial 27 with value: 0.886.
/tmp/ipython-input-3095368924.py:95: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.712400,0.382720,0.838000,0.841797,0.811676,0.874239


[I 2025-11-08 08:19:32,487] Trial 29 pruned. 



--- Random Search Complete ---

BEST HYPERPARAMETERS FOUND:
BestRun(run_id='27', objective=0.886, hyperparameters={'learning_rate': 4.282224740784025e-05, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 2, 'weight_decay': 0.07, 'num_train_epochs': 2}, run_summary=None)

Best Hyperparameters:
  learning_rate: 4.282224740784025e-05
  per_device_train_batch_size: 8
  gradient_accumulation_steps: 2
  weight_decay: 0.07
  num_train_epochs: 2


AttributeError: 'BestRun' object has no attribute 'metrics'